## Dependencies

---

## Pre-loading

---

In [4]:
import pandas as pd

### Functions:

In [12]:
def add_uid(func):
    """
    Add uid to the dimension created.
    :param func: function that creates dimensions.
    :return: the dimension created by the function + an uid.
    """
    def wrapper(**kargs) -> pd.DataFrame:
        dim = func(**kargs)
        dim[f"{kargs['column']}_id"] = dim.index + 1
        return dim
    return wrapper

In [13]:
@add_uid
def create_dim(column:str, dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Extract the feature and parse it to a dataframe.
    :param column: feature to extract.
    :param dataframe: object where feature will be extracted.
    :return: dataframe with the feature and its new column.
    """
    dim = dataframe[f"{column}"].to_frame()
    return dim

### Dimensions & facts:

First of all is required to split the dataframes in `Dimensions`, where they can be related with the `Facts` by an unique id.

In [14]:
df_temp = pd.read_csv("persistence/sedan-cards_march-april.csv")

In [17]:
df_temp.sample(5)

,model,year,kilometers,engine,price,description
152,Mitsubishi Mirage G4,2021,4000,Gasolina,10000,Color : Negro Bonito Mitsubishi Mirage G4 2021...
117,Ford Focus,2008,141,Gasolina,3500,Color : Negro DESCRIPCIÓN ARTÍCULOS CALIFICACI...
29,Chevrolet Spark,2016,75005,Gasolina,7500,Garantía: Como es visto no hay garantía\nFinan...
149,Mitsubishi Mirage G4,2021,3000,Gasolina,12000,Precio negociable: no Color : Gris Vendo Mitsu...
72,Mercedes Benz 200,1988,195000,Gasolina,4000,Garantía: Como es visto no hay garantía Precio...


In [56]:
columns = df_temp.columns.to_list()
dimensions = ["model", "description","engine"]
facts = ["year","kilometers","price"]

In [57]:
create_dim_uid = create_dim

In [58]:
for column in columns:
    if column in dimensions and columns in facts:
        print("a")
    elif column in dimensions:
        print("d")
    elif column in facts:
        print("f")
    else:
        print("no one")

d
f
f
d
f
d


In [9]:
print(len(dimensions), len(facts))

3 3
